In [ ]:
preprocess_1

In [ ]:
前期处理
比如将特征的类型转换，便于后边算法训练用

In [ ]:
716修正：增加orderdate的minute，后边训练可以选择要不要用。
但是先增加上。

In [ ]:
718修正：id类的特征，先不要删去。后边说不定要用
——要不然second（秒）也加上得了。

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_x = pd.read_csv('dataset/train/ord_train_update.csv',encoding = 'GB2312')
test_A = pd.read_csv('dataset/test/ord_testA_update.csv',encoding = 'GB2312')
test_B = pd.read_csv('dataset/test/ord_testB_update.csv',encoding = 'GB2312')

In [3]:
train_x.shape,test_A.shape,test_B.shape

((4023086, 24), (11035, 23), (11036, 23))

In [ ]:
先把train_x按照order日期排序

In [4]:
train_x.sort_values(['orderdate'],inplace = True)

In [ ]:
将三者合并

In [5]:
train_and_test = pd.concat([train_x,test_A,test_B])

In [6]:
index_split_train_and_test = 4023086
index_split_train_and_testB = 4034121

In [ ]:
index位置
train 0-4023085
testA 4023086-4034120
testB 4034121-4045156

In [7]:
train_and_test.shape

(4045157, 24)

In [12]:
train_and_test.iloc[4023085:4023088,:]

,arrival,city,countryid,etd,glat,glon,hotel,hotelbelongto,hotelstar,isebookinghtl,...,noroom,ordadvanceday,orderdate,orderid,room,supplierchannel,supplierid,wuli_totalrooms,zifangxing_totalrooms,zone
684457,2017-10-20,248,78,2017-10-21,33.591500,130.419020,7044764,SHT,0,T,...,0.0,36,2017-09-14 23:59:52,4504252505,61072586,合作,3685.0,110.0,7.0,NaN
0,2017/10/11,633,66,2017/10/15,40.687355,-73.982550,6265739,HTL,3,T,...,NaN,27,2017-09-15 14:11:00,4505896718,71340604,直签,2299.0,670.0,4.0,-1.0
1,2017/9/19,410,42,2017/9/20,37.446570,126.456345,2811117,SHT,3,F,...,NaN,4,2017-09-15 04:32:00,4504466510,70578904,合作,3536.0,7.0,58.0,NaN


In [13]:
train_and_test.iloc[4034120:4034122,:]

,arrival,city,countryid,etd,glat,glon,hotel,hotelbelongto,hotelstar,isebookinghtl,...,noroom,ordadvanceday,orderdate,orderid,room,supplierchannel,supplierid,wuli_totalrooms,zifangxing_totalrooms,zone
11034,2017/10/16,3916,2,2017/10/17,2.550903,101.796936,5224606,SHT,0,F,...,NaN,31,2017-09-15 12:33:00,4505455290,36691160,合作,4577.0,226.0,22.0,NaN
0,2017/9/18,364,32,2017/9/22,14.532218,120.986206,6546241,SHT,4,T,...,NaN,3,2017-09-15 17:36:00,4506801369,78296910,合作,4489.0,365.0,100.0,13226.0


In [ ]:
date类型转换（没法保存，必须每次都做）

In [8]:
from dateutil.parser import parse
train_and_test['orderdate'] = train_and_test['orderdate'].apply(parse)

In [ ]:
object类型的转换（cat化）

In [9]:
train_and_test['supplierchannel'].replace('合作','hezuo',inplace = True)
train_and_test['supplierchannel'].replace('直签','zhiqian',inplace = True)
train_and_test['supplierchannel'].replace('集团直连','jituanzhilian',inplace = True)

In [ ]:
object列，必须变成categorical的类型（或者是float）
注意这里arrival和etd作为时间，不变成cat更合理

In [10]:
train_and_test['supplierchannel'] = train_and_test['supplierchannel'].astype('category')
train_and_test['isholdroom'] = train_and_test['isholdroom'].astype('category')
train_and_test['hotelbelongto'] = train_and_test['hotelbelongto'].astype('category')
train_and_test['isebookinghtl'] = train_and_test['isebookinghtl'].astype('category')

In [11]:
train_and_test['supplierchannel'] = train_and_test['supplierchannel'].cat.codes
train_and_test['isholdroom'] = train_and_test['isholdroom'].cat.codes
train_and_test['hotelbelongto'] = train_and_test['hotelbelongto'].cat.codes
train_and_test['isebookinghtl'] = train_and_test['isebookinghtl'].cat.codes

In [ ]:
month和day不要，hour保留：
原因：test只有1天。
arrival和etd还是要的，但是order的day就不要了

In [13]:
train_and_test.iloc[0,16].second

3

In [ ]:
这次把minute和second也都加上。

In [12]:
hour = []
minute = []
for item in train_and_test['orderdate']:
    hour.append(item.hour)
    minute.append(item.minute)

train_and_test['order_hour'] = hour
train_and_test['order_minute'] = minute

In [ ]:
arrival和std的处理方式不太一样，因为没变成date，还是str
——同时train和test部分格式还不一样，需要分别处理

In [13]:
day = []
count = 0
for item in train_and_test['arrival']:
    if count < index_split_train_and_test:
        day.append(int(item[5:7] + item[8:]))
    else:
        day.append(int(item[5:].replace('/','')))  
    count = count + 1

train_and_test['arrival_day'] = day

In [14]:
day = []
count = 0
for item in train_and_test['etd']:
    if count < index_split_train_and_test:
        day.append(int(item[5:7] + item[8:]))
    else:
        day.append(int(item[5:].replace('/','')))  
    count = count + 1

train_and_test['etd_day'] = day

In [15]:
# 这个留到后边train时决定如何处理
# train_and_test.drop(['arrival','etd'],axis = 1 ,inplace = True)

In [18]:
train_and_test.shape

(4045157, 28)

In [25]:
train_and_test.head()

,arrival,city,countryid,etd,glat,glon,hotel,hotelbelongto,hotelstar,isebookinghtl,...,supplierchannel,supplierid,wuli_totalrooms,zifangxing_totalrooms,zone,order_hour,order_minute,order_second,arrival_day,etd_day
1008396,2017-02-06,220,9,2017-02-07,25.130505,55.116665,3015862,3,5,1,...,3,2322.0,841.0,172.0,NaN,0,0,3,206,207
3541880,2017-03-31,359,4,2017-04-01,13.728729,100.527115,6409984,3,3,1,...,3,1897.0,928.0,5.0,381.0,0,0,16,331,401
227988,2017-02-01,9052,83,2017-02-02,7.929710,80.720260,227845,1,5,0,...,1,2294.0,36.0,142.0,8263.0,0,0,19,201,202
1666191,2017-02-15,92573,78,2017-02-20,26.220081,127.675960,5722491,3,3,1,...,3,4657.0,214.0,8.0,1199.0,0,0,28,215,220
2631949,2017-02-22,35725,78,2017-02-25,26.450941,127.803370,1932564,3,4,1,...,3,2364.0,3.0,31.0,1196.0,0,0,31,222,225


In [19]:
train_and_test.columns

Index(['arrival', 'city', 'countryid', 'etd', 'glat', 'glon', 'hotel',
       'hotelbelongto', 'hotelstar', 'isebookinghtl', 'isholdroom', 'isvendor',
       'masterbasicroomid', 'masterhotelid', 'noroom', 'ordadvanceday',
       'orderdate', 'orderid', 'room', 'supplierchannel', 'supplierid',
       'wuli_totalrooms', 'zifangxing_totalrooms', 'zone', 'order_hour',
       'order_minute', 'arrival_day', 'etd_day'],
      dtype='object')

In [ ]:
分别保存

In [16]:
test_A = train_and_test.iloc[index_split_train_and_test:index_split_train_and_testB,:]
test_B = train_and_test.iloc[index_split_train_and_testB:,:]
train_x = train_and_test.iloc[:index_split_train_and_test,:]

In [17]:
test_A.shape,test_B.shape,train_x.shape

((11035, 28), (11036, 28), (4023086, 28))

In [18]:
train_x.to_csv('dataset/train/ord_train_pre1.csv',index = False)
test_A.to_csv('dataset/test/ord_testA_pre1.csv',index = False)
test_B.to_csv('dataset/test/ord_testB_pre1.csv',index = False)

In [19]:
import numpy as np
import pandas as pd

In [20]:
train_x = pd.read_csv('dataset/train/ord_train_pre1.csv',encoding = 'GB2312')
test_A = pd.read_csv('dataset/test/ord_testA_pre1.csv',encoding = 'GB2312')
test_B = pd.read_csv('dataset/test/ord_testB_pre1.csv',encoding = 'GB2312')

In [ ]:
继续处理，开始设计特征

In [3]:
train_x.head()

,arrival,city,countryid,etd,glat,glon,hotel,hotelbelongto,hotelstar,isebookinghtl,...,supplierchannel,supplierid,wuli_totalrooms,zifangxing_totalrooms,zone,order_hour,order_minute,order_second,arrival_day,etd_day
0,2017-02-06,220,9,2017-02-07,25.130505,55.116665,3015862,3,5,1,...,3,2322.0,841.0,172.0,NaN,0,0,3,206,207
1,2017-03-31,359,4,2017-04-01,13.728729,100.527115,6409984,3,3,1,...,3,1897.0,928.0,5.0,381.0,0,0,16,331,401
2,2017-02-01,9052,83,2017-02-02,7.929710,80.720260,227845,1,5,0,...,1,2294.0,36.0,142.0,8263.0,0,0,19,201,202
3,2017-02-15,92573,78,2017-02-20,26.220081,127.675960,5722491,3,3,1,...,3,4657.0,214.0,8.0,1199.0,0,0,28,215,220
4,2017-02-22,35725,78,2017-02-25,26.450941,127.803370,1932564,3,4,1,...,3,2364.0,3.0,31.0,1196.0,0,0,31,222,225


In [25]:
train_x.columns

Index(['arrival', 'city', 'countryid', 'etd', 'glat', 'glon', 'hotel',
       'hotelbelongto', 'hotelstar', 'isebookinghtl', 'isholdroom', 'isvendor',
       'masterbasicroomid', 'masterhotelid', 'noroom', 'ordadvanceday',
       'orderdate', 'orderid', 'room', 'supplierchannel', 'supplierid',
       'wuli_totalrooms', 'zifangxing_totalrooms', 'zone', 'order_hour',
       'order_minute', 'arrival_day', 'etd_day'],
      dtype='object')

In [ ]:
需要处理的特征：
1 arrival和etd不要，同名的day也不要
添加：停留时长 = etd - arrival + 1
——————718的修正：这些id先保留。可以以后不要。但是先保留！————————
2 添加了glat和glon后，母酒店id可以由剩下的几个特征唯一确定（根据母酒店table），因此可以删除masterhotelid
3 物理房型由个数wuli room决定，masterbasicroomid可以删除
4 子房型的hotel也可以由其他特征来决定，可以删除

In [ ]:
处理1

In [21]:
from dateutil.parser import parse
train_x['arrival'] = train_x['arrival'].apply(parse)
train_x['etd'] = train_x['etd'].apply(parse)
test_A['arrival'] = test_A['arrival'].apply(parse)
test_A['etd'] = test_A['etd'].apply(parse)
test_B['arrival'] = test_B['arrival'].apply(parse)
test_B['etd'] = test_B['etd'].apply(parse)

In [22]:
day_stay = []
for i in range(train_x.shape[0]):
    temp = train_x.iloc[i,3] - train_x.iloc[i,0]
    day_stay.append(temp)

In [23]:
for i in range(len(day_stay)):
    day_stay[i] = day_stay[i].days

In [24]:
day_stay[:4]

[1, 1, 1, 5]

In [25]:
train_x['day_stay'] = day_stay

In [ ]:
删除无用信息

In [26]:
drop_list = ['arrival',  'etd',  'hotel','masterbasicroomid', 'masterhotelid','arrival_day', 'etd_day']
# drop_list = ['arrival',  'etd','arrival_day', 'etd_day']

In [27]:
train_x.drop(drop_list,axis = 1,inplace = True)

In [28]:
train_x.shape

(4023086, 22)

In [ ]:
两个test

In [29]:
day_stay = []
for i in range(test_A.shape[0]):
    temp = test_A.iloc[i,3] - test_A.iloc[i,0]
    day_stay.append(temp)

In [30]:
for i in range(len(day_stay)):
    day_stay[i] = day_stay[i].days

In [31]:
test_A['day_stay'] = day_stay

In [32]:
test_A.drop(drop_list,axis = 1,inplace = True)

In [33]:
test_A.shape

(11035, 22)

In [34]:
day_stay = []
for i in range(test_B.shape[0]):
    temp = test_B.iloc[i,3] - test_B.iloc[i,0]
    day_stay.append(temp)

In [35]:
for i in range(len(day_stay)):
    day_stay[i] = day_stay[i].days

In [36]:
test_B['day_stay'] = day_stay

In [37]:
test_B.drop(drop_list,axis = 1,inplace = True)

In [38]:
test_B.shape

(11036, 22)

In [ ]:
保存

In [39]:
train_x.to_csv('dataset/train/ord_train_pre2.csv',index = False)
test_A.to_csv('dataset/test/ord_testA_pre2.csv',index = False)
test_B.to_csv('dataset/test/ord_testB_pre2.csv',index = False)

In [ ]:
716的修正
增加orderdate的minute（上边已经添加了）

In [2]:
train_x = pd.read_csv('dataset/train/ord_train_pre2.csv')
test_A = pd.read_csv('dataset/test/ord_testA_pre2.csv',encoding = 'GB2312')
test_B = pd.read_csv('dataset/test/ord_testB_pre2.csv',encoding = 'GB2312')

In [3]:
from dateutil.parser import parse
train_x['orderdate'] = train_x['orderdate'].apply(parse)
test_A['orderdate'] = test_A['orderdate'].apply(parse)
test_B['orderdate'] = test_B['orderdate'].apply(parse)

In [4]:
minute = []
for item in train_x['orderdate']:
    minute.append(item.minute)

train_x['order_minute'] = minute

minute = []
for item in test_A['orderdate']:
    minute.append(item.minute)

test_A['order_minute'] = minute

minute = []
for item in test_B['orderdate']:
    minute.append(item.minute)

test_B['order_minute'] = minute

In [5]:
train_x.shape,test_A.shape

((4023086, 22), (11035, 22))

In [6]:
train_x.to_csv('dataset/train/ord_train_pre2.csv',index = False)
test_A.to_csv('dataset/test/ord_testA_pre2.csv',index = False)
test_B.to_csv('dataset/test/ord_testB_pre2.csv',index = False)

In [46]:
train_x.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotel', 'hotelbelongto',
       'hotelstar', 'isebookinghtl', 'isholdroom', 'isvendor',
       'masterbasicroomid', 'masterhotelid', 'noroom', 'ordadvanceday',
       'orderdate', 'orderid', 'room', 'supplierchannel', 'supplierid',
       'wuli_totalrooms', 'zifangxing_totalrooms', 'zone', 'order_hour',
       'order_minute', 'day_stay'],
      dtype='object')